In [2]:
### Importacion de librerias
import hashlib
import time

In [3]:
### Definicion de funcion para hashear una cadenas de texto

def hash_tx(tx):
  hash_object = hashlib.sha256(tx.encode())
  hash_hex = hash_object.hexdigest()
  return hash_hex

In [4]:
### se crea la funcion de arbol de merkel

def create_markle_root(transactions):
  hashed_transactions = [hash_tx(tx) for tx in transactions]

  while len(hashed_transactions) > 1:
    if len(hashed_transactions) % 2 != 0:
      hashed_transactions.append(hashed_transactions[-1])
    new_lvl = []
    for i in range(0, len(hashed_transactions), 2):
      concatenate_hasheds = hashed_transactions[i] + hashed_transactions[i + 1]
      new_hash = hash_tx(concatenate_hasheds)
      new_lvl.append(new_hash)

    hashed_transactions = new_lvl
  return hashed_transactions

In [5]:
class Block:
  def __init__(self, previous_hash, transactions, miner_address):
    self.previous_hash = previous_hash
    self.transactions = ["Coinbase transaction: reward to " + miner_address] + transactions
    self.timestamp = time.time()
    self.merkle_root = create_markle_root(transactions)
    self.nonce = 0
    self.hash = self.calculate_hash()

  def calculate_hash(self):
    block_content = str(self.previous_hash) + str(self.merkle_root) + str(self.timestamp) + str(self.nonce)
    return hash_tx(block_content)

  def mine_block(self, difficulty):
    required_prefix = "0" * difficulty
    while not self.hash.startswith(required_prefix):
      self.nonce += 1
      self.hash = self.calculate_hash()
    print("Bloque minado:" + self.hash)

In [6]:
### Lista de transacciones

transactions1 = [
    "Transacción 1",
    "Transacción 2",
    "Transacción 3",
    "Transacción 4"
]

transactions2 = [
    "Transacción 5",
    "Transacción 6",
    "Transacción 7",
    "Transacción 8"
]

In [7]:
### creacion del bloque genesis
### genesis_block = Block("0", transactions1, "Miner")

In [8]:
###Imprime las vairables dentro del block genesis
### print(vars(genesis_block))

In [9]:
###Calcula el hash bloque genesis
### genesis_block.mine_block(5)
### print(genesis_block.hash)

In [31]:
class Blockchain:
  def __init__(self, difficulty):
    self.chain = []  # Lista que contendrá todos los bloques
    self.difficulty = difficulty # Puedes ajustar la dificultad según lo necesites

    # Crear el bloque génesis (el primer bloque de la cadena)
    genesis_block = Block("0", ["Genesis Block"], "Miner")
    genesis_block.mine_block(self.difficulty)
    self.chain.append(genesis_block)

  def get_latest_block(self):
    # Retorna el último bloque de la cadena
    return self.chain[-1]

  def add_block(self, new_block):
    # Establecer el hash del bloque anterior
    new_block.previous_hash = self.get_latest_block().hash

    # Minar el nuevo bloque
    new_block.mine_block(self.difficulty)

    if self.validate_transactions(new_block.transactions):
      self.chain.append(new_block)
    else:
      print("Transacciones inválidas. El bloque no se ha agregado.")

  def validate_transactions(self, transactions):

  # Valida las transacciones en el bloque.
  # En este ejemplo, la validación simplemente verifica que no haya transacciones duplicadas.
    transaction_set = set()
    for tx in transactions:
        if tx in transaction_set:
            print("Transacción duplicada detectada:", tx)
            return False
        transaction_set.add(tx)
    return True

  def is_chain_valid(self):
    # Verifica la integridad de la cadena de bloques
    for i in range(1, len(self.chain)):
        current_block = self.chain[i]
        previous_block = self.chain[i - 1]

        # Verificar que el hash actual es correcto
        if current_block.hash != current_block.calculate_hash():
            print("Hash incorrecto en el bloque:", i)
            return False

        # Verificar que el bloque apunta correctamente al bloque anterior
        if current_block.previous_hash != previous_block.hash:
            print("Enlace roto entre bloques:", i)
            return False

    return True


In [41]:
# Crear una instancia de la blockchain
my_blockchain = Blockchain(5)

Bloque minado:000006225a849e4eafee17d9d76d9462be8c35dda4cb7bc18fa52673478ec903


In [42]:
# Agregar nuevos bloques a la blockchain
new_block_1 = Block(my_blockchain.get_latest_block().hash, transactions1,"wallet1")
my_blockchain.add_block(new_block_1)

Bloque minado:0000001c8215a4131329b1b6caf6074e8926b05fe3e8a3724c5af353c4f653d2


In [44]:
# Agregar nuevos bloques a la blockchain
new_block_2 = Block(my_blockchain.get_latest_block().hash, transactions2, "wallet2")
my_blockchain.add_block(new_block_2)

Bloque minado:0000085776d7fdac0ed36310d928f84eb095b42347dad0f00c4eff8233014652


In [46]:
# Verificar la validez de la cadena
print("¿La cadena es válida?", my_blockchain.is_chain_valid())

¿La cadena es válida? True


In [45]:
for i, block in enumerate(my_blockchain.chain):
    print(f"Bloque {i} - Hash: {block.hash} - Hash Anterior: {block.previous_hash}")


Bloque 0 - Hash: 000006225a849e4eafee17d9d76d9462be8c35dda4cb7bc18fa52673478ec903 - Hash Anterior: 0
Bloque 1 - Hash: 0000001c8215a4131329b1b6caf6074e8926b05fe3e8a3724c5af353c4f653d2 - Hash Anterior: 000006225a849e4eafee17d9d76d9462be8c35dda4cb7bc18fa52673478ec903
Bloque 2 - Hash: 0000096fbedde2ad843fcdc88d5337a9148ff3d130398fdc8f24b9d5f6bbdfa4 - Hash Anterior: 0000001c8215a4131329b1b6caf6074e8926b05fe3e8a3724c5af353c4f653d2
Bloque 3 - Hash: 0000085776d7fdac0ed36310d928f84eb095b42347dad0f00c4eff8233014652 - Hash Anterior: 0000096fbedde2ad843fcdc88d5337a9148ff3d130398fdc8f24b9d5f6bbdfa4
